In [4]:
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import time
from datetime import timedelta
import h5py
from tqdm import tqdm
import argparse
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

import sys

ProjectPath = "/home/liranc6/ecg/ecg_forecasting" #os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
sys.path.append(ProjectPath)  # Add the parent directory to the sys.path

import liran_project.utils.dataset_loader as dataset_loader

# import liran_project.train as liran_train
from liran_project.utils.dataset_loader import SingleLeadECGDatasetCrops
from liran_project.utils.util import modify_z_and_omega

sys.path.append('/home/liranc6/ecg/ecg_forecasting/SSSD_main')

from SSSD_main.src.utils.util import find_epoch, print_size, sampling, calc_diffusion_hyperparams
from SSSD_main.src.utils.util import get_mask_mnr, get_mask_bm, get_mask_rm, get_mask_pred

import SSSD_main.src.utils.util as util
from SSSD_main.src.imputers.DiffWaveImputer import DiffWaveImputer
from SSSD_main.src.imputers.SSSDSAImputer import SSSDSAImputer
from SSSD_main.src.imputers.SSSDS4Imputer import SSSDS4Imputer

from sklearn.metrics import mean_squared_error
from statistics import mean

ModuleNotFoundError: No module named 'liran_project'

In [1]:
file_path = \
"/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/SSSDS4/context248_label1248/T200_beta00.0001_betaT0.02/best_model:_iter:_1_loss:_0.9853200713793436.pth"

In [6]:
# open file and get the keys
checkpoint = torch.load(file_path, map_location='cpu')
print(checkpoint.keys())
checkpoint["wandb_id"]

dict_keys(['model_state_dict', 'optimizer_state_dict', 'wandb_id'])


'kdnog64l'

In [13]:
dict_path = \
"/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/SSSDS4/context3748_label2500/T200_beta00.0001_betaT0.02"

In [14]:
import torch

for root, dirs, files in os.walk(dict_path):
    found = False
    for file in files:
        if file.endswith(".pth"):
            file_path = os.path.join(root, file)
            model_checkpoint = torch.load(file_path)
            if "wandb_id" in model_checkpoint:
                print(model_checkpoint["wandb_id"])
                if model_checkpoint["wandb_id"] == "f7y4e71z":
                    print("found the file")
                    print(file_path)
                    found = True
                    break
                else:
                    #skip to next directory
                    break
                    
            else:
                print("not found")
    if found:
        break

212frl4y
4sd0opjk


In [24]:
import wandb
import pandas as pd

# Authenticate with the API
api = wandb.Api()

# Define your project and run details
entity = 'liranc6'  # your username or team name
project = 'ecg_SSSD_SSSDS4'
run_id = 'ywwhgg9f'

# Fetch the run
run = api.run(f'{entity}/{project}/{run_id}')
print(run)  # Debugging line

# Fetch the history of the run
history = run.history()
print(history)  # Debugging line

# Convert the history to a DataFrame for easier manipulation
df = pd.DataFrame(history)
print(df)  # Debugging line

# Check if "iteration" is a column in the DataFrame
print(df.columns)  # Debugging line

print(df["iteration"].notna().sum())

<Run liranc6/ecg_SSSD_SSSDS4/ywwhgg9f (failed)>
     train_loss_per_t    _timestamp mean_extra_r_beats   _step dtw_dist  \
0             0.29767  1.720549e+09               None     385     None   
1             0.17208  1.720549e+09               None     459     None   
2             0.13548  1.720549e+09               None     727     None   
3             0.04812  1.720549e+09               None     741     None   
4             0.05236  1.720549e+09               None    1085     None   
..                ...           ...                ...     ...      ...   
495           0.00686  1.720747e+09               None  165773     None   
496           0.00675  1.720747e+09               None  166268     None   
497           0.00508  1.720747e+09               None  166437     None   
498           0.00767  1.720747e+09               None  166700     None   
499           0.00775  1.720747e+09               None  166923     None   

     validation_loss_per_t mean_extra_r_beats_negli

In [ ]:

# Determine the last 10 log entries to delete
N = 10  # Number of log entries to delete
new_df = df.iloc[:-N]  # Exclude the last N log entries

# Start a new run
new_run = wandb.init(project=project, entity=entity, name=f'{run.name}_updated')

# Log the filtered data to the new run
for index, row in new_df.iterrows():
    row_dict = row.to_dict()
    new_run.log(row_dict)

# Finish the new run
new_run.finish()

print("Created a new run without the last 10 log entries.")